# Exploratory Data Analysis

We see 9 different files, each with numerous transactions along with other metadata.

Let us drill down and see what we can find. 

In [2]:
import pandas as pd
import glob

import matplotlib.pyplot as plt


import seaborn as sns

import datetime
import numpy as np

Importing files

In [3]:
files = glob.glob("data/*.csv")

In [4]:
columns = ['sku', 'brand', 'cat', 'bill', 'store', 'date', 'type', 'mrp', 'price', 'qty']

The 'source' column below is a marker to indicate file origin of the observation 

In [5]:
#Helper function to read multiple csv files and store it in a dictionary. Found in helper.py

from helper import read_files

In [6]:
df_dict = read_files(files, columns)

### Question - The data is split into 9 files. Does it mean anything? 

In [7]:
#Helper function to test for overlap in the different files. Found in helper.py

from helper import overlap

In [8]:
#A few random checks for overlaps

overlap(df_dict['df1'], df_dict['df2'], 'sku')

'Overlaps found for sku'

In [9]:
overlap(df_dict['df1'], df_dict['df6'], 'brand')

'Overlaps found for brand'

In [10]:
overlap(df_dict['df4'], df_dict['df7'], 'bill')

'There is no overlap'

### It appears that the transactions are put into different files on the basis of different bill numbers

We assume that other than this, there is not much difference between the transactions in the files.

Hence, we are going to concatenate all the 9 files into one big file. 

In [13]:
df = pd.concat(df for df in df_dict.values())

# Data cleaning and formatting

In [15]:
df['store'] = df['store'].apply(lambda x: x[-1])

### Handling missing values

In [17]:
df.isna().sum()

sku        0
brand     13
cat        9
bill       0
store      0
date       0
type       0
mrp        0
price      0
qty        0
source     0
dtype: int64

### Only brand and cat columns have missing values. Let's investigate further. 

In [19]:
# Subset where brand is NaN

df[df['brand'].isnull()]

,sku,brand,cat,bill,store,date,type,mrp,price,qty,source
10,SKU0726,NaN,Cat01,BILL02585,6,2017-07-19,Sale,54.67,54.67,1,9
11,SKU0726,NaN,Cat01,BILL02585,6,2017-07-19,Sale,54.67,54.67,1,9
29,SKU2585,NaN,NaN,BILL09133,3,2017-09-09,Sale,2.60,2.60,1,4
1278,SKU2649,NaN,NaN,BILL09643,3,2017-09-06,Sale,8.00,8.00,1,4
1372,SKU2653,NaN,NaN,BILL09736,3,2017-09-07,Sale,30.00,30.00,1,4
1373,SKU2649,NaN,NaN,BILL09737,3,2017-09-07,Sale,8.00,8.00,1,4
1434,SKU2654,NaN,NaN,BILL09771,3,2017-09-07,Sale,9.67,9.67,1,4
1478,SKU2657,NaN,NaN,BILL09798,3,2017-09-08,Sale,3.73,0.00,1,4
1482,SKU2658,NaN,NaN,BILL09802,3,2017-09-08,Sale,4.33,4.33,1,4
1490,SKU2659,NaN,NaN,BILL09809,3,2017-09-08,Sale,4.33,4.33,1,4


In [20]:
### Instead of throwing the data, we replace missing brands by 'UnkBRAND' and missing categories by 'UnkCAT'

,sku,brand,cat,bill,store,date,type,mrp,price,qty,source
29,SKU2585,NaN,NaN,BILL09133,3,2017-09-09,Sale,2.60,2.60,1,4
1278,SKU2649,NaN,NaN,BILL09643,3,2017-09-06,Sale,8.00,8.00,1,4
1372,SKU2653,NaN,NaN,BILL09736,3,2017-09-07,Sale,30.00,30.00,1,4
1373,SKU2649,NaN,NaN,BILL09737,3,2017-09-07,Sale,8.00,8.00,1,4
1434,SKU2654,NaN,NaN,BILL09771,3,2017-09-07,Sale,9.67,9.67,1,4
1478,SKU2657,NaN,NaN,BILL09798,3,2017-09-08,Sale,3.73,0.00,1,4
1482,SKU2658,NaN,NaN,BILL09802,3,2017-09-08,Sale,4.33,4.33,1,4
1490,SKU2659,NaN,NaN,BILL09809,3,2017-09-08,Sale,4.33,4.33,1,4
1506,SKU2661,NaN,NaN,BILL09825,3,2017-09-08,Sale,3.99,3.99,1,4


In [ ]:
df.loc[df['brand'].isnull(), 'brand'] = 'UnkBRAND'

df.loc[df['cat'].isnull(), 'cat'] = 'UnkCAT'